In [1]:
import numpy as np
import pandas as pd

In [2]:
aisles = pd.read_csv('data/aisles.csv')
departments = pd.read_csv('data/departments.csv')
order_products__prior = pd.read_csv('data/order_products__prior.csv')
order_products__train = pd.read_csv('data/order_products__train.csv')
orders = pd.read_csv('data/orders.csv')
products = pd.read_csv('data/products.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

In [3]:
# given prior, predict things in test (scoring https://www.kaggle.com/c/instacart-market-basket-analysis#evaluation)
# cluster different buy patterns for items and customers

In [36]:
orders_train = orders[orders.eval_set == 'train']
orders_train['prediction'] = 'None'

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [63]:
def aggregate_rows(rows):
    if sum(rows.reordered) == 0:
        return ('None')
    filter_products = [str(product) for reordered, product in zip(rows.reordered, rows.product_id) if reordered]
    return (filter_products)

grouped = order_products__train.groupby(order_products__train.order_id)
products_by_order = grouped.apply(func=aggregate_rows)
real_orders = products_by_order.sort_index().values
predicted_orders = [(prediction) for prediction in orders_train.sort('order_id').prediction.values]

def score_row(real_order, predicted_order):
    correct_elements = float(len([product for product in predicted_order if product in real_order]))
    precision = correct_elements / len(predicted_order)
    recall = correct_elements / len(real_order)
    return 2*(precision * recall) / (precision + recall) if correct_elements else 0

score = sum(score_row(real, predicted) for real, predicted in zip(real_orders, predicted_orders)) / len(real_orders)

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [64]:
score

0.06555952716658156